In [9]:
import pandas as pd
import os,json

In [10]:
# need to clean up this block of code
csv_dir="logs"
program_file_dir="."
# get all program files that end with .py
program_files = [f for f in os.listdir(program_file_dir) if f.endswith(".py")]
program = program_files[0].split(".")[0]
same_program_csv_dir=os.path.join(csv_dir,program)
# samples = total number of runs for each program
samples = len(os.listdir(same_program_csv_dir))
op_to_func = {}

# loop through csv files in csv_dir
for program_ in program_files:
    program = program_.split(".")[0]
    same_program_csv_dir=os.path.join(csv_dir,program)
    common_funcs_per_op = set()
    c_func_to_count = {}
    for csv_file in os.listdir(same_program_csv_dir):
        df = pd.read_csv(os.path.join(same_program_csv_dir,csv_file))
        # merge two columns with "|" as separator
        df["full_func"] = df['Function'].astype(str) + "|" + df['Module'].astype(str)
        # loop through each function in the column 'full_func'
        for func in df["full_func"].values:
            # if function is not in the dictionary, add it
            if func not in c_func_to_count:
                c_func_to_count[func] = 1
            # else increment the count
            else:
                c_func_to_count[func] += 1

    for func in c_func_to_count:
        # i.e. present in more than half of the samples
        # if c_func_to_count[func] > samples//2:
        common_funcs_per_op.add(func)        

    op_to_func[program] = list(common_funcs_per_op)
    print(f"Common functions for {program} are {common_funcs_per_op}")

Common functions for RandomHorizontalFlip are {'__libc_calloc|libc.so.6', '__GI___pthread_mutex_lock|libpthread.so.0', '_PyEval_EvalFrameDefault|python3.10', 'ImagingFlipLeftRight|_imaging.cpython-310-x86_64-linux-gnu.so', '__GI___libc_malloc|libc.so.6', 'c10::impl::OperatorEntry::lookup|libtorch_cpu.so'}
Common functions for Loader are {'__libc_calloc|libc.so.6', 'read|python3.10', '__memmove_avx_unaligned_erms|libc.so.6', '_int_free|libc.so.6', '__GI___libc_malloc|libc.so.6', 'PyEval_RestoreThread|python3.10', 'vgetargs1_impl|python3.10', '__memset_avx2_unaligned_erms|libc.so.6', 'ImagingJpegDecode|_imaging.cpython-310-x86_64-linux-gnu.so', 'PyVectorcall_Function|python3.10', 'decode_mcu|libjpeg.so.9', 'PyThread_get_thread_ident|python3.10', 'ycc_rgb_convert|libjpeg.so.9', '_Py_NewReference|python3.10', '__libc_read|libpthread.so.0', 'ImagingUnpackRGB|_imaging.cpython-310-x86_64-linux-gnu.so', 'jpeg_idct_16x16|libjpeg.so.9', 'decompress_onepass|libjpeg.so.9', 'jpeg_read_scanlines|lib

In [11]:
op_to_func

{'RandomHorizontalFlip': ['__libc_calloc|libc.so.6',
  '__GI___pthread_mutex_lock|libpthread.so.0',
  '_PyEval_EvalFrameDefault|python3.10',
  'ImagingFlipLeftRight|_imaging.cpython-310-x86_64-linux-gnu.so',
  '__GI___libc_malloc|libc.so.6',
  'c10::impl::OperatorEntry::lookup|libtorch_cpu.so'],
 'Loader': ['__libc_calloc|libc.so.6',
  'read|python3.10',
  '__memmove_avx_unaligned_erms|libc.so.6',
  '_int_free|libc.so.6',
  '__GI___libc_malloc|libc.so.6',
  'PyEval_RestoreThread|python3.10',
  'vgetargs1_impl|python3.10',
  '__memset_avx2_unaligned_erms|libc.so.6',
  'ImagingJpegDecode|_imaging.cpython-310-x86_64-linux-gnu.so',
  'PyVectorcall_Function|python3.10',
  'decode_mcu|libjpeg.so.9',
  'PyThread_get_thread_ident|python3.10',
  'ycc_rgb_convert|libjpeg.so.9',
  '_Py_NewReference|python3.10',
  '__libc_read|libpthread.so.0',
  'ImagingUnpackRGB|_imaging.cpython-310-x86_64-linux-gnu.so',
  'jpeg_idct_16x16|libjpeg.so.9',
  'decompress_onepass|libjpeg.so.9',
  'jpeg_read_scanline

In [12]:

def find_common_cpp_func(op_to_func):
    # find functions found in other ops
    common_funcs_to_op = {}
    # loop through all programs
    for python_func in op_to_func.keys():
        # loop through all functions in the program
        for func in op_to_func[python_func]:
            # loop through all other programs
            for other_python_func in op_to_func.keys():
                # skip if the program is the same as the current program
                if other_python_func == python_func:
                    continue
                # if the function is found in other programs
                if func in op_to_func[other_python_func]:
                    # add the function to the set of common functions
                    if func not in common_funcs_to_op:
                        common_funcs_to_op[func] = [python_func,other_python_func]
                    elif python_func not in common_funcs_to_op[func] or other_python_func not in common_funcs_to_op[func]:
                        common_funcs_to_op[func].append(other_python_func)
    return common_funcs_to_op

In [13]:
common_funcs_to_op = find_common_cpp_func(op_to_func)
print("C/C++ functions that can be found in multiple python operations:")
for func in common_funcs_to_op.keys():
    print(f"\t{func}\n\t\t->\t{common_funcs_to_op[func]}")

C/C++ functions that can be found in multiple python operations:
	__libc_calloc|libc.so.6
		->	['RandomHorizontalFlip', 'Loader']
	_PyEval_EvalFrameDefault|python3.10
		->	['RandomHorizontalFlip', 'ToTensor']
	__GI___libc_malloc|libc.so.6
		->	['RandomHorizontalFlip', 'Loader', 'ToTensor']
	__memmove_avx_unaligned_erms|libc.so.6
		->	['Loader', 'RandomResizedCrop', 'ToTensor']
	_int_free|libc.so.6
		->	['Loader', 'RandomResizedCrop', 'Collation', 'ToTensor']
	vgetargs1_impl|python3.10
		->	['Loader', 'ToTensor']
	_Py_NewReference|python3.10
		->	['Loader', 'ToTensor']
	c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::native::AVX2::VectorizedLoop2d<at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(float)#1}, at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(at::vec::AVX2

In [14]:
json_string = {'op_to_func':op_to_func,'common_funcs_to_op':common_funcs_to_op}
with open('mapping_funcs.json', 'w') as outfile:
    json.dump(json_string, outfile,indent=4)